## Google api to obtain the distance between the stadium

Please note use your own google api key

In [ ]:
import googlemaps
import pandas as pd

#stadium dataframe
stadiums = pd.read_csv("stadium_df.csv")
stadium_list = stadiums['stadium'].tolist()

gmaps = googlemaps.Client(key='YOUR-API-KEY')

#create a matrix of distances between stadiums
distance_matrix = []
for stadium in stadium_list:
    distance_matrix.append(gmaps.distance_matrix(stadium_list, stadium)['rows'][0]['elements'])

#convert distance matrix to dataframe
distance_df = pd.DataFrame(distance_matrix)

#add stadium names to dataframe
distance_df.columns = stadium_list
distance_df['stadium'] = stadium_list

#remove mi from distance
distance_df = distance_df.replace(to_replace=' mi', value='', regex=True)

#convert distance to float
distance_df = distance_df.astype(float)

#if distance is less than 1, convert to 0
distance_df[distance_df < 1] = 0

## Bing Api to obtain the shortest drivable distance between the stadium

Please note use your own bing api key. Also this will take 5 minutes to run as it will obtain the distances between the stadiums.

In [1]:
import requests

# Define the base URL for the Bing Maps API
base_url = "http://dev.virtualearth.net/REST/v1/Routes"


# Stadiums and their coordinates (extracted from your file)
stadiums = ["State Farm Stadium",
"Allegiant Stadium",
"AT&T Stadium",
"Raymond James Stadium",
"NRG Stadium",
"Mercedes-Benz Stadium",
"Nissan Stadium",
"Lincoln Financial Field",
"Gillette Stadium",
"MetLife Stadium",
"Soldier Field",
"Ford Field",
"Acrisure Stadium",
"U.S. Bank Stadium",
"Paycor Stadium",
"GEHA Field at Arrowhead Stadium",
"Empower Field at Mile High",
"Lumen Field",
"Levi's Stadium",
"Sofi Stadium"] 

coordinates = [(33.5276, -112.2626),
(36.0907, -115.1831),
(32.7473, -97.0945),
(27.9759, -82.5033),
(29.6847, -95.4107),
(33.7554, -84.4008),
(36.1665, -86.7713),
(39.9008, -75.1675),
(42.0909, -71.2643),
(40.8128, -74.0742),
(41.8623, -87.6167),
(42.34, -83.0456),
(42.9758, -85.6756),
(44.9738, -93.258),
(39.0954, -84.516),
(39.0489, -94.4839),
(39.7439, -105.0201),
(47.5952, -122.3316),
(37.403, -121.9702),
(33.9534, -118.3392)]

# Function to format coordinates
def format_coords(lat, lng):
    return f"{lat},{lng}"

# Initialize an empty distance matrix
distance_matrix = []

# Loop through the stadiums to calculate distances
for idx1, (lat1, lng1) in enumerate(coordinates):
    row = []
    for idx2, (lat2, lng2) in enumerate(coordinates):
        if idx1 == idx2:
            row.append(0.0)
        else:
            params = {
                "wayPoint.1": format_coords(lat1, lng1),
                "wayPoint.2": format_coords(lat2, lng2),
                "travelMode": "Driving",
                "optimize": "time",
                "distanceUnit": "mi",
                "dateTime": "11/18/2023 06:00:00",
                "timeType": "Departure",
                "key": "Please enter you key",
            }
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                route = data["resourceSets"][0]["resources"][0]
                route_distance = route["travelDistance"]
                row.append(route_distance)
            else:
                row.append(None)
    distance_matrix.append(row)

# Print or process the distance matrix as needed
print(distance_matrix)


[[0.0, 276.041667, 1063.688062, 2172.407515, 1196.050068, 1861.922001, 1652.944927, 2359.824873, 2625.411969, 2420.799406, 1766.709293, 1999.039982, 1916.296951, 1657.613288, 1819.51031, 1223.642677, 821.518581, 1392.879956, 709.02989, 377.080972], [280.37076, 0.0, 1208.28176, 2323.942551, 1479.385389, 1962.579163, 1794.83131, 2484.00342, 2719.849827, 2526.244234, 1758.016931, 2019.741584, 1916.127938, 1663.625676, 1943.688857, 1362.695608, 754.092351, 1123.158256, 526.389013, 277.331012], [1063.960222, 1203.957637, 0.0, 1123.256432, 266.265012, 799.801658, 684.4093, 1480.434264, 1769.729778, 1565.117215, 942.896609, 1203.983735, 1092.484267, 995.013496, 950.857617, 520.52824, 778.828516, 2067.004941, 1682.881348, 1431.020341], [2171.436933, 2321.773965, 1123.176897, 0.0, 989.179442, 458.832294, 705.801246, 1037.756999, 1332.046474, 1127.433911, 1173.605519, 1180.17714, 1248.806346, 1586.46753, 917.883311, 1251.609351, 1860.274124, 3094.185581, 2870.445971, 2538.497052], [1194.433881, 

In [2]:
print("Distance Matrix (in kilometers):")
header = [""] + stadiums
print("\t".join(header))
for i in range(len(stadiums)):
    row = [stadiums[i]] + distance_matrix[i]
    print("\t".join(map(str, row)))


Distance Matrix (in kilometers):
	State Farm Stadium	Allegiant Stadium	AT&T Stadium	Raymond James Stadium	NRG Stadium	Mercedes-Benz Stadium	Nissan Stadium	Lincoln Financial Field	Gillette Stadium	MetLife Stadium	Soldier Field	Ford Field	Acrisure Stadium	U.S. Bank Stadium	Paycor Stadium	GEHA Field at Arrowhead Stadium	Empower Field at Mile High	Lumen Field	Levi's Stadium	Sofi Stadium
State Farm Stadium	0.0	276.041667	1063.688062	2172.407515	1196.050068	1861.922001	1652.944927	2359.824873	2625.411969	2420.799406	1766.709293	1999.039982	1916.296951	1657.613288	1819.51031	1223.642677	821.518581	1392.879956	709.02989	377.080972
Allegiant Stadium	280.37076	0.0	1208.28176	2323.942551	1479.385389	1962.579163	1794.83131	2484.00342	2719.849827	2526.244234	1758.016931	2019.741584	1916.127938	1663.625676	1943.688857	1362.695608	754.092351	1123.158256	526.389013	277.331012
AT&T Stadium	1063.960222	1203.957637	0.0	1123.256432	266.265012	799.801658	684.4093	1480.434264	1769.729778	1565.117215	942.896

## For consitency we will be using the distance file obtained from the google api

In [6]:
import pandas as pd
stadiums = ["State Farm Stadium",
"Allegiant Stadium",
"AT&T Stadium",
"Raymond James Stadium",
"NRG Stadium",
"Mercedes-Benz Stadium",
"Nissan Stadium",
"Lincoln Financial Field",
"Gillette Stadium",
"MetLife Stadium",
"Soldier Field",
"Ford Field",
"Acrisure Stadium",
"U.S. Bank Stadium",
"Paycor Stadium",
"GEHA Field at Arrowhead Stadium",
"Empower Field at Mile High",
"Lumen Field",
"Levi's Stadium",
"Sofi Stadium"] 


# Read the CSV file
distance_matrix = pd.read_csv("distance_df.csv")


In [7]:
distances = distance_matrix.iloc[:,1:].to_numpy()
distances

array([[   0. ,  280. , 1063. , 2174. , 1196. , 1822. , 1652. , 2360. ,
        2659. , 2421. , 1766. , 2000. , 2072. , 1656. , 1819. , 1224. ,
         821. , 1405. ,  710. ,  379. ],
       [ 283. ,    0. , 1209. , 2327. , 1465. , 1964. , 1795. , 2484. ,
        2719. , 2524. , 1757. , 2020. , 2199. , 1663. , 1943. , 1363. ,
         754. , 1131. ,  526. ,  278. ],
       [1064. , 1209. ,    0. , 1124. ,  265. ,  799. ,  684. , 1479. ,
        1778. , 1564. ,  942. , 1203. , 1238. ,  992. ,  950. ,  560. ,
         780. , 2078. , 1683. , 1433. ],
       [2173. , 2326. , 1123. ,    0. ,  990. ,  459. ,  705. , 1039. ,
        1334. , 1128. , 1172. , 1180. , 1027. , 1588. ,  917. , 1251. ,
        1860. , 3094. , 2800. , 2543. ],
       [1197. , 1465. ,  265. ,  992. ,    0. ,  802. ,  789. , 1549. ,
        1848. , 1634. , 1089. , 1308. , 1343. , 1187. , 1055. ,  751. ,
        1036. , 2334. , 1898. , 1566. ],
       [1830. , 1972. ,  800. ,  460. ,  801. ,    0. ,  248. ,  772. ,
   

## MTZ Base Model

In [8]:
import gurobipy as gp
from gurobipy import *
import numpy as np


# Create a model
model = gp.Model("TSP")

# Create binary decision variables for TSP
n = len(stadiums)
x = model.addVars(n, n, vtype=GRB.BINARY, name='x')

# Continuous variables for MTZ constraints
u = model.addVars(n, lb=1, ub=n-1, vtype=GRB.CONTINUOUS, name='u')

# Set objective to minimize total distance
model.setObjective(gp.quicksum(distances[i, j] * x[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

# Add constraint to ensure each location is visited once
for i in range(n):
    model.addConstr(quicksum(x[i, j] for j in range(n) if j != i) == 1)

# Add constraint to ensure each location is left once
for i in range(n):
    model.addConstr(quicksum(x[j, i] for j in range(n) if j != i) == 1)

# Add MTZ subtour elimination constraints
for i in range(2, n):
    for j in range(2, n):
        if i != j:
            model.addConstr(u[i] - u[j] + n * x[i, j] <= n - 1)
# Optimize the model
model.optimize()

# Print the optimal tour
tour = []
for i in range(n):
    for j in range(n):
        if x[i, j].X > 0.5:
            tour.append((stadiums[i], stadiums[j]))

print("Optimal Tour:")
for i, j in tour:
    print(f"From {i} to {j}")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 5900HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads



Optimize a model with 346 rows, 420 columns and 1678 nonzeros
Model fingerprint: 0xd0c28d68
Variable types: 20 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+02, 3e+03]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 22 columns
Presolve time: 0.00s
Presolved: 346 rows, 398 columns, 1678 nonzeros
Variable types: 18 continuous, 380 integer (380 binary)
Found heuristic solution: objective 20084.000000
Found heuristic solution: objective 19510.000000

Root relaxation: objective 7.912600e+03, 66 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7912.60000    0   29 19510.0000 7912.60000  59.4%     -    0s
H    0     0                    15253.000000 7912.60000  48.1%     -    0s
H    0     0                    15137.00

In [9]:
# Assuming 'tour' is a list of tuples representing the optimal tour

next_dest = {}
for origin, dest in tour:
    next_dest[origin] = dest

# A function to print the trip from a given origin stadium
def print_trip(origin):
    # Loop until there is no next destination or a circular reference is detected
    visited = set()
    while origin in next_dest:
        # Get the next destination
        dest = next_dest[origin]
        # Print the arrow and the destination
        print(origin + " -> " + dest)
        # Update the origin
        origin = dest
        # Mark the origin as visited
        visited.add(origin)
        if dest == starting_stadium:  # Replace with the name of the starting stadium
            break
    # Print a new line
    print()

# Replace 'starting_stadium' with the name of the stadium you want to start the tour from
starting_stadium = "MetLife Stadium"
print("The optimal distance in miles is:", model.objVal)
print_trip(starting_stadium)



The optimal distance in miles is: 9692.4
MetLife Stadium -> Lincoln Financial Field
Lincoln Financial Field -> Acrisure Stadium
Acrisure Stadium -> Paycor Stadium
Paycor Stadium -> Nissan Stadium
Nissan Stadium -> Mercedes-Benz Stadium
Mercedes-Benz Stadium -> Raymond James Stadium
Raymond James Stadium -> NRG Stadium
NRG Stadium -> AT&T Stadium
AT&T Stadium -> State Farm Stadium
State Farm Stadium -> Allegiant Stadium
Allegiant Stadium -> Sofi Stadium
Sofi Stadium -> Levi's Stadium
Levi's Stadium -> Lumen Field
Lumen Field -> Empower Field at Mile High
Empower Field at Mile High -> GEHA Field at Arrowhead Stadium
GEHA Field at Arrowhead Stadium -> U.S. Bank Stadium
U.S. Bank Stadium -> Soldier Field
Soldier Field -> Ford Field
Ford Field -> Gillette Stadium
Gillette Stadium -> MetLife Stadium



In [10]:
print("The optimal distance in miles is:", model.objVal)


The optimal distance in miles is: 9692.4


## Fischetti and Toth Model


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
from itertools import product

# Define callback function for subtour elimination
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get solution values
        vals = model.cbGetSolution(model._vars)
        # Find subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

# Function to find subtour in the solution
def subtour(vals):
    # Create a list of edges based on values in vals
    edges = [(i, j) for i, j in product(range(n), range(n)) if vals[i, j] > 0.5]
    # Create a list of unvisited nodes
    unvisited = list(range(n))
    # Initialize the cycle with indices from 0 to n
    cycle = range(n+1)
    # Start a loop while there are unvisited nodes
    while unvisited:
        # Initialize an empty list for the current cycle
        thiscycle = []
        # Start another loop while there are neighbors to explore
        neighbors = unvisited
        while neighbors:
            # Get the first neighbor
            current = neighbors[0]
            # Add the current node to the current cycle
            thiscycle.append(current)
            # Remove the current node from the list of unvisited nodes
            unvisited.remove(current)
            # Update the neighbors to be the nodes connected to the current node
            neighbors = [j for i, j in edges if i == current and j in unvisited]
        # Check if the current cycle is shorter than the existing cycle
        if len(thiscycle) <= len(cycle):
            # Update the cycle if the current cycle is shorter
            cycle = thiscycle
    # Return the final cycle
    return cycle

# Read in the stadiums and distances data
stadiums_df = pd.read_csv("stadium_df.csv")
distances_df = pd.read_csv("distance_df.csv")

# Convert the stadiums DataFrame to a list of tuples [(lat1, lon1), (lat2, lon2), ..., (latN, lonN)]
stadiums = list(zip(stadiums_df['lat'], stadiums_df['lng']))

# Reset the first column as the index in distances distances_df
distances_df = distances_df.set_index(distances_df.columns[0])

# Convert the distances DataFrame to a 2D list (distance matrix)
dist_matrix = distances_df.values.tolist()

# Assuming the stadium names are in a column called "Name"
stadium_names = stadiums_df['stadium'].tolist()

n = len(stadiums)  # Number of stadiums: 20

# Create model
m = gb.Model('ATSP')

# Add decision variables
vars = {}
for i, j in product(range(n), range(n)):
    vars[i, j] = m.addVar(vtype=GRB.BINARY, name='e' + str(i) + '_' + str(j))
m.update()

# Add objective function
m.setObjective(gb.quicksum(dist_matrix[i][j] * vars[i, j] for i, j in product(range(n), range(n))), GRB.MINIMIZE)

# Add constraints
m.addConstrs(vars[i, i] == 0 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for j in range(n)) == 1 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for i in range(n)) == 1 for j in range(n))

# Set up lazy constraints for subtour elimination
m._vars = vars
m.params.LazyConstraints = 1

# Optimize
m.optimize(subtour_elimination)

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', vars)

    # Manually set the starting point to State Farm Stadium
    start_stadium_name = "State Farm Stadium"
    start_stadium_index = stadium_names.index(start_stadium_name)

    # Find the starting point of the tour
    start_stadium = start_stadium_index

    # Check if the starting stadium is valid
    if start_stadium is not None:
        # Initialize the current stadium as the starting point
        current_stadium = start_stadium

        # Create a list to store the sequential travel information
        sequential_travel = []

        # Iterate through the tour
        while True:
            # Find the next stadium in the tour
            next_stadium = None
            for j in range(n):
                if solution[current_stadium, j] > 0.5:
                    next_stadium = j
                    break

            # Append the travel information to the list
            sequential_travel.append((stadium_names[current_stadium], stadium_names[next_stadium]))

            # Update the current stadium for the next iteration
            current_stadium = next_stadium

            # Check if the tour is complete
            if current_stadium == start_stadium:
                break

        # Print the sequential travel information
        for i, (from_stadium, to_stadium) in enumerate(sequential_travel):
            print(f"Travel {i+1} from {from_stadium} to {to_stadium}")

    else:
        print(f"Starting stadium {start_stadium_name} not found. Please check the input data or model constraints.")
else:
    print("No optimal solution found")

# Print objective value
print(f"Total distance traveled: {m.objVal}")

# Dispose of the model object to avoid memory leaks
m.dispose()

# Count the number of stadiums visited
count = 0
for i in range(n):
    for j in range(n):
        if solution[i, j] > 0.5:
            count += 1
print(f"Number of stadiums visited: {count}")


## F&T No starting point

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
from itertools import product

from itertools import chain

# Define callback function for subtour elimination
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get solution values
        vals = model.cbGetSolution(model._vars)
        # Find subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        # Find the subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour(vals):
    # Create a list of edges based on values in vals
    edges = [(i, j) for i, j in product(range(n), range(n)) if vals[i, j] > 0.5]
    # Create a list of unvisited nodes
    unvisited = list(range(n))
    # Initialize the cycle with indices from 0 to n
    cycle = range(n+1)
    # Start a loop while there are unvisited nodes
    while unvisited:
        # Initialize an empty list for the current cycle
        thiscycle = []
        # Start another loop while there are neighbors to explore
        neighbors = unvisited
        while neighbors:
            # Get the first neighbor
            current = neighbors[0]
            # Add the current node to the current cycle
            thiscycle.append(current)
            # Remove the current node from the list of unvisited nodes
            unvisited.remove(current)
            # Update the neighbors to be the nodes connected to the current node
            neighbors = [j for i, j in edges if i == current and j in unvisited]
        # Check if the current cycle is shorter than the existing cycle
        if len(thiscycle) <= len(cycle):
            # Update the cycle if the current cycle is shorter
            cycle = thiscycle
    # Return the final cycle
    return cycle

# Read in the stadiums and distances data
stadiums_df = pd.read_csv("stadium_df.csv")
distances_df = pd.read_csv("distance_df.csv")

# Convert the stadiums DataFrame to a list of tuples [(lat1, lon1), (lat2, lon2), ..., (latN, lonN)]
stadiums = list(zip(stadiums_df['lat'], stadiums_df['lng']))

# Reset the first column as index in distances distances_df
distances_df = distances_df.set_index(distances_df.columns[0])

# Convert the distances DataFrame to a 2D list (distance matrix)
dist_matrix = distances_df.values.tolist()

# Assuming the stadium names are in a column called "Name"
stadium_names = stadiums_df['stadium'].tolist()

n = len(stadiums)  # Number of stadiums: 20

# Create model
m = gb.Model('ATSP')

# Add decision variables
vars = {}
for i, j in product(range(n), range(n)):
    vars[i, j] = m.addVar(vtype=GRB.BINARY, name='e'+str(i)+'_'+str(j))
m.update()

# Add objective function
m.setObjective(gb.quicksum(dist_matrix[i][j]*vars[i, j] for i, j in product(range(n), range(n))), GRB.MINIMIZE)

# Add constraints
m.addConstrs(vars[i, i] == 0 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for j in range(n)) == 1 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for i in range(n)) == 1 for j in range(n))

# Set up lazy constraints for subtour elimination
m._vars = vars
m.params.LazyConstraints = 1

# Optimize
m.optimize(subtour_elimination)

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', vars)

    # Find the starting point of the tour
    start_stadium = None
    for i in range(n):
        in_degree = sum(solution[j, i] for j in range(n))
        if in_degree == 0:
            start_stadium = i
            break

    # Check if a starting stadium is found
    if start_stadium is not None:
        # Initialize the current stadium as the starting point
        current_stadium = start_stadium

        # Iterate through the tour
        while True:
            # Find the next stadium in the tour
            next_stadium = None
            for j in range(n):
                if solution[current_stadium, j] > 0.5:
                    next_stadium = j
                    break

            # Print the travel information
            print(f"Travel from {stadium_names[current_stadium]} to {stadium_names[next_stadium]}")

            # Update the current stadium for the next iteration
            current_stadium = next_stadium

            # Check if the tour is complete
            if current_stadium == start_stadium:
                break
    else:
        print("No starting stadium found. Please check the input data or model constraints.")
else:
    print("No optimal solution found")

# Print objective value
print(f"Total distance traveled: {m.objVal}")

# Dispose of the model object to avoid memory leaks
m.dispose()

# Count the number of stadiums visited
count = 0
for i in range(n):
    for j in range(n):
        if solution[i, j] > 0.5:
            count += 1
print(f"Number of stadiums visited: {count}")

## If the tour starts from Raymond James Stadium

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
from itertools import product

from itertools import chain

# Define callback function for subtour elimination
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get solution values
        vals = model.cbGetSolution(model._vars)
        # Find subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        # Find the subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour(vals):
    # Create a list of edges based on values in vals
    edges = [(i, j) for i, j in product(range(n), range(n)) if vals[i, j] > 0.5]
    # Create a list of unvisited nodes
    unvisited = list(range(n))
    # Initialize the cycle with indices from 0 to n
    cycle = range(n+1)
    # Start a loop while there are unvisited nodes
    while unvisited:
        # Initialize an empty list for the current cycle
        thiscycle = []
        # Start another loop while there are neighbors to explore
        neighbors = unvisited
        while neighbors:
            # Get the first neighbor
            current = neighbors[0]
            # Add the current node to the current cycle
            thiscycle.append(current)
            # Remove the current node from the list of unvisited nodes
            unvisited.remove(current)
            # Update the neighbors to be the nodes connected to the current node
            neighbors = [j for i, j in edges if i == current and j in unvisited]
        # Check if the current cycle is shorter than the existing cycle
        if len(thiscycle) <= len(cycle):
            # Update the cycle if the current cycle is shorter
            cycle = thiscycle
    # Return the final cycle
    return cycle

# Read in the stadiums and distances data
stadiums_df = pd.read_csv("stadium_df.csv")
distances_df = pd.read_csv("distance_df.csv")

# Convert the stadiums DataFrame to a list of tuples [(lat1, lon1), (lat2, lon2), ..., (latN, lonN)]
stadiums = list(zip(stadiums_df['lat'], stadiums_df['lng']))

# Reset the first column as the index in distances distances_df
distances_df = distances_df.set_index(distances_df.columns[0])

# Convert the distances DataFrame to a 2D list (distance matrix)
dist_matrix = distances_df.values.tolist()

# Assuming the stadium names are in a column called "Name"
stadium_names = stadiums_df['stadium'].tolist()

n = len(stadiums)  # Number of stadiums: 20

# Create model
m = gb.Model('ATSP')

# Add decision variables
vars = {}
for i, j in product(range(n), range(n)):
    vars[i, j] = m.addVar(vtype=GRB.BINARY, name='e' + str(i) + '_' + str(j))
m.update()

# Add objective function
m.setObjective(gb.quicksum(dist_matrix[i][j] * vars[i, j] for i, j in product(range(n), range(n))), GRB.MINIMIZE)

# Add constraints
m.addConstrs(vars[i, i] == 0 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for j in range(n)) == 1 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for i in range(n)) == 1 for j in range(n))

# Set up lazy constraints for subtour elimination
m._vars = vars
m.params.LazyConstraints = 1

# Optimize
m.optimize(subtour_elimination)

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', vars)

    # Manually set the starting point to Raymond James Stadium
    start_stadium_name = "Raymond James Stadium"
    start_stadium_index = stadium_names.index(start_stadium_name)

    # Find the starting point of the tour
    start_stadium = start_stadium_index

    # Check if the starting stadium is valid
    if start_stadium is not None:
        # Initialize the current stadium as the starting point
        current_stadium = start_stadium

        # Iterate through the tour
        while True:
            # Find the next stadium in the tour
            next_stadium = None
            for j in range(n):
                if solution[current_stadium, j] > 0.5:
                    next_stadium = j
                    break

            # Print the travel information
            print(f"Travel from {stadium_names[current_stadium]} to {stadium_names[next_stadium]}")

            # Update the current stadium for the next iteration
            current_stadium = next_stadium

            # Check if the tour is complete
            if current_stadium == start_stadium:
                break
    else:
        print(f"Starting stadium {start_stadium_name} not found. Please check the input data or model constraints.")
else:
    print("No optimal solution found")

# Print objective value
print(f"Total distance traveled: {m.objVal}")

# Dispose of the model object to avoid memory leaks
m.dispose()

# Count the number of stadiums visited
count = 0
for i in range(n):
    for j in range(n):
        if solution[i, j] > 0.5:
            count += 1
print(f"Number of stadiums visited: {count}")

## If the tour starts from MetLife Stadium

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *

# Define callback function for subtour elimination
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get solution values
        vals = model.cbGetSolution(model._vars)
        # Find subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

# Function to find subtour in the solution
def subtour(vals):
    # Create a list of edges based on values in vals
    edges = [(i, j) for i, j in product(range(n), range(n)) if vals[i, j] > 0.5]
    # Create a list of unvisited nodes
    unvisited = list(range(n))
    # Initialize the cycle with indices from 0 to n
    cycle = range(n+1)
    # Start a loop while there are unvisited nodes
    while unvisited:
        # Initialize an empty list for the current cycle
        thiscycle = []
        # Start another loop while there are neighbors to explore
        neighbors = unvisited
        while neighbors:
            # Get the first neighbor
            current = neighbors[0]
            # Add the current node to the current cycle
            thiscycle.append(current)
            # Remove the current node from the list of unvisited nodes
            unvisited.remove(current)
            # Update the neighbors to be the nodes connected to the current node
            neighbors = [j for i, j in edges if i == current and j in unvisited]
        # Check if the current cycle is shorter than the existing cycle
        if len(thiscycle) <= len(cycle):
            # Update the cycle if the current cycle is shorter
            cycle = thiscycle
    # Return the final cycle
    return cycle

# Read in the stadiums and distances data
stadiums_df = pd.read_csv("stadium_df.csv")
distances_df = pd.read_csv("distance_df.csv")

# Convert the stadiums DataFrame to a list of tuples [(lat1, lon1), (lat2, lon2), ..., (latN, lonN)]
stadiums = list(zip(stadiums_df['lat'], stadiums_df['lng']))

# Reset the first column as the index in distances distances_df
distances_df = distances_df.set_index(distances_df.columns[0])

# Convert the distances DataFrame to a 2D list (distance matrix)
dist_matrix = distances_df.values.tolist()

# Assuming the stadium names are in a column called "Name"
stadium_names = stadiums_df['stadium'].tolist()

n = len(stadiums)  # Number of stadiums: 20

# Create model
m = gb.Model('ATSP')

# Add decision variables
vars = {}
for i, j in product(range(n), range(n)):
    vars[i, j] = m.addVar(vtype=GRB.BINARY, name='e' + str(i) + '_' + str(j))
m.update()

# Add objective function
m.setObjective(gb.quicksum(dist_matrix[i][j] * vars[i, j] for i, j in product(range(n), range(n))), GRB.MINIMIZE)

# Add constraints
m.addConstrs(vars[i, i] == 0 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for j in range(n)) == 1 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for i in range(n)) == 1 for j in range(n))

# Set up lazy constraints for subtour elimination
m._vars = vars
m.params.LazyConstraints = 1

# Optimize
m.optimize(subtour_elimination)

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', vars)

    # Manually set the starting point to MetLife Stadium
    start_stadium_name = "MetLife Stadium"
    start_stadium_index = stadium_names.index(start_stadium_name)

    # Find the starting point of the tour
    start_stadium = start_stadium_index

    # Check if the starting stadium is valid
    if start_stadium is not None:
        # Initialize the current stadium as the starting point
        current_stadium = start_stadium

        # Iterate through the tour
        while True:
            # Find the next stadium in the tour
            next_stadium = None
            for j in range(n):
                if solution[current_stadium, j] > 0.5:
                    next_stadium = j
                    break

            # Print the travel information
            print(f"Travel from {stadium_names[current_stadium]} to {stadium_names[next_stadium]}")

            # Update the current stadium for the next iteration
            current_stadium = next_stadium

            # Check if the tour is complete
            if current_stadium == start_stadium:
                break
    else:
        print(f"Starting stadium {start_stadium_name} not found. Please check the input data or model constraints.")
else:
    print("No optimal solution found")

# Print objective value
print(f"Total distance traveled: {m.objVal}")

# Dispose of the model object to avoid memory leaks
m.dispose()

# Count the number of stadiums visited
count = 0
for i in range(n):
    for j in range(n):
        if solution[i, j] > 0.5:
            count += 1
print(f"Number of stadiums visited: {count}")

## Original Code

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
from itertools import product

from itertools import chain

# Define callback function for subtour elimination
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get solution values
        vals = model.cbGetSolution(model._vars)
        # Find subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        # Find the subtour
        tour = subtour(vals)
        if len(tour) < n:
            # Add subtour elimination constraint
            model.cbLazy(quicksum(model._vars[i,j] for i in tour for j in tour if i != j) <= len(tour)-1)

def subtour(vals):
    # Create a list of edges based on values in vals
    edges = [(i, j) for i, j in product(range(n), range(n)) if vals[i, j] > 0.5]
    # Create a list of unvisited nodes
    unvisited = list(range(n))
    # Initialize the cycle with indices from 0 to n
    cycle = range(n+1)
    # Start a loop while there are unvisited nodes
    while unvisited:
        # Initialize an empty list for the current cycle
        thiscycle = []
        # Start another loop while there are neighbors to explore
        neighbors = unvisited
        while neighbors:
            # Get the first neighbor
            current = neighbors[0]
            # Add the current node to the current cycle
            thiscycle.append(current)
            # Remove the current node from the list of unvisited nodes
            unvisited.remove(current)
            # Update the neighbors to be the nodes connected to the current node
            neighbors = [j for i, j in edges if i == current and j in unvisited]
        # Check if the current cycle is shorter than the existing cycle
        if len(thiscycle) <= len(cycle):
            # Update the cycle if the current cycle is shorter
            cycle = thiscycle
    # Return the final cycle
    return cycle

# Read in the stadiums and distances data
stadiums_df = pd.read_csv("/stadium_df.csv")
distances_df = pd.read_csv("distance_df.csv")

# Convert the stadiums DataFrame to a list of tuples [(lat1, lon1), (lat2, lon2), ..., (latN, lonN)]
stadiums = list(zip(stadiums_df['lat'], stadiums_df['lng']))

# Reset the first column as index in distances distances_df
distances_df = distances_df.set_index(distances_df.columns[0])

# Convert the distances DataFrame to a 2D list (distance matrix)
dist_matrix = distances_df.values.tolist()

# Assuming the stadium names are in a column called "Name"
stadium_names = stadiums_df['stadium'].tolist()

n = len(stadiums)  # Number of stadiums: 20

# Create model
m = gb.Model('ATSP')
m = gb.Model('ATSP')

# Add decision variables
vars = {}
for i, j in product(range(n), range(n)):
    vars[i, j] = m.addVar(vtype=GRB.BINARY, name='e'+str(i)+'_'+str(j))
m.update()

# Add objective function
m.setObjective(gb.quicksum(dist_matrix[i][j]*vars[i, j] for i, j in product(range(n), range(n))), GRB.MINIMIZE)

# Add constraints
m.addConstrs(vars[i, i] == 0 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for j in range(n)) == 1 for i in range(n))
m.addConstrs(gb.quicksum(vars[i, j] for i in range(n)) == 1 for j in range(n))

# Set up lazy constraints for subtour elimination
m._vars = vars
m.params.LazyConstraints = 1

# Optimize
m.optimize(subtour_elimination)

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', vars)
    for i in range(n):
        for j in range(n):
            if solution[i, j] > 0.5:
                print(f"Travel from {stadium_names[i]} to {stadium_names[j]}")
else:
    print("No optimal solution found")

# Print objective value
print(f"Total distance traveled: {m.objVal}")

# Dispose of the model object to avoid memory leaks
m.dispose()

# Count the number of stadiums visited
count = 0
for i in range(n):
    for j in range(n):
        if solution[i, j] > 0.5:
            count += 1
print(f"Number of stadiums visited: {count}")